In [1]:
import os
import string
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

from langchain_huggingface import HuggingFaceEmbeddings
tqdm.pandas()

# Loading dataset

In [2]:
# Diretório dos arquivos
diretorio = "data/full/sentences/"

dados = []

for nome_arquivo in os.listdir(diretorio):
    caminho = os.path.join(diretorio, nome_arquivo)
    if os.path.isfile(caminho):
        data = nome_arquivo.replace(".txt", "")  # remove extensão se houver
        with open(caminho, "r", encoding="utf-8") as f:
            for linha in f:
                sentenca = linha.strip()
                if sentenca:
                    dados.append({"data": data, "sentenca": sentenca})


df = pd.DataFrame(dados)
df["data"] = pd.to_datetime(df["data"], format="%d%m%Y", errors="coerce")
print(df.head())

        data                                           sentenca
0 2019-12-11  Ata da 22 7ª Reunião do Comitê de Política Mon...
1 2019-12-11  Banco Central do Brasil 227ª Reunião do Copom ...
2 2019-12-11  Dados de atividade econômica a partir do segun...
3 2019-12-11  O cenário do Copom supõe que essa recuperação ...
4 2019-12-11  Os dados sugerem que a economia segue operando...


In [4]:
df

,data,sentenca
0,2019-12-11,Ata da 22 7ª Reunião do Comitê de Política Mon...
1,2019-12-11,Banco Central do Brasil 227ª Reunião do Copom ...
2,2019-12-11,Dados de atividade econômica a partir do segun...
3,2019-12-11,O cenário do Copom supõe que essa recuperação ...
4,2019-12-11,Os dados sugerem que a economia segue operando...
...,...,...
55930,2001-10-17,Saiba mais na nossa.
55931,2001-10-17,Gerenciar cookies.
55932,2001-10-17,Rejeitar cookies.
55933,2001-10-17,Aceitar cookies.


In [5]:
# Gerar embeddings usando modelo HuggingFace
embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

In [6]:
# Gera o vetor da palavra "inflação"
vetor_inflacao = embeddings.embed_query("inflação")
vetor_inflacao_alta = embeddings.embed_query("alta na inflação")
vetor_inflacao_baixa = embeddings.embed_query("baixa na inflação")

In [7]:
# Função para calcular a distância (cosine similarity invertida)
def calcular_distancia(sentenca, vetor_referencia):
    vetor_sentenca = embeddings.embed_query(sentenca)
    # Cosine similarity retorna valor entre -1 e 1; quanto mais próximo de 1, mais similar
    similaridade = cosine_similarity([vetor_sentenca], [vetor_referencia])[0][0]
    # Para transformar em "distância", usamos 1 - similaridade
    distancia = 1 - similaridade
    return distancia

In [8]:
df["inflation"] = df["sentenca"].progress_apply(lambda x: calcular_distancia(x, vetor_inflacao))

100%|██████████| 55935/55935 [12:02<00:00, 77.46it/s]


In [9]:
df["high_inflation"] = df["sentenca"].progress_apply(lambda x: calcular_distancia(x, vetor_inflacao_alta))

100%|██████████| 55935/55935 [12:02<00:00, 77.37it/s]


In [10]:
df["low_inflation"] = df["sentenca"].progress_apply(lambda x: calcular_distancia(x, vetor_inflacao_baixa))

100%|██████████| 55935/55935 [12:02<00:00, 77.38it/s]


In [12]:
df.to_csv('data/sentences_qwen_06b.csv', index=False)

In [14]:
df

,data,sentenca,inflation,high_inflation,low_inflation
0,2019-12-11,Ata da 22 7ª Reunião do Comitê de Política Mon...,0.563261,0.630031,0.556545
1,2019-12-11,Banco Central do Brasil 227ª Reunião do Copom ...,0.606990,0.645778,0.574385
2,2019-12-11,Dados de atividade econômica a partir do segun...,0.647879,0.677341,0.585323
3,2019-12-11,O cenário do Copom supõe que essa recuperação ...,0.692430,0.687794,0.668582
4,2019-12-11,Os dados sugerem que a economia segue operando...,0.637811,0.689423,0.580398
...,...,...,...,...,...
55930,2001-10-17,Saiba mais na nossa.,0.615611,0.602023,0.626810
55931,2001-10-17,Gerenciar cookies.,0.614764,0.606718,0.609366
55932,2001-10-17,Rejeitar cookies.,0.568679,0.573838,0.547600
55933,2001-10-17,Aceitar cookies.,0.548883,0.534904,0.558244


In [23]:
# filter df with high_inflation < 0.5 or low_inflation < 0.5
filtered_df = df[(df['high_inflation'] < 0.5) | (df['low_inflation'] < 0.5)]
print('Quantidade de sentenças resultantes:', len(filtered_df),'\n\n')
for x in filtered_df['sentenca'].head(20):
    print(x)
    print('\n----\n')

Quantidade de sentenças resultantes: 2376 


Diversas medidas de inflação subjacente encontram-se em níveis confortáveis, inclusive os componentes mais sensíveis ao ciclo econômico e à política monetária.

----

B) Riscos em torno do cenário básico para a inflação 8.

----

Alguns membros avaliaram que os últimos dados de atividade e a maior eficiência do mercado de crédito e capitais podem resultar em uma redução da ociosidade mais rápida do que a ntevisto e, com isso, produzir pressão altista na inflação.

----

No horizonte relevante para a política monetária, o cenário com taxa Selic extraída da pesquisa Focus e taxa de câmbio constante produz inflação ligeiramente abaixo da meta para 2020 e ao redor da meta para 2021.

----

O Co pom avalia que a conjuntura econômica com expectativas de inflação ancoradas, medidas de inflação subjacente em níveis confortáveis, cenários com projeções para a inflação abaixo ou ao redor da meta para o horizonte relevante de política monetária e eleva

In [24]:
filtered_df = df[(df['high_inflation'] < 0.6) | (df['low_inflation'] < 0.6)]
print('Quantidade de sentenças resultantes:', len(filtered_df),'\n\n')
for x in filtered_df['sentenca'].head(20):
    print(x)
    print('\n----\n')

Quantidade de sentenças resultantes: 18723 


Ata da 22 7ª Reunião do Comitê de Política Monetária (Copom) do Banco Central do Brasil 10 e 11 de dezembro de 201 9 Banco Central do Brasil 227ª Reunião do Copom 2 Data: 10 e 11/12/2019 Local: Sala s de reuniões do 8º andar (10/12) e do 20º andar ( 11/12) do Edifício-Sede do Banco Central do Brasil – Brasília – DF Horário de início e de término das sessões: 10/12: 10h10 – 13h04; 15h13 – 19h38 11/12: 14h30 – 18h11 Presentes: Membros do Copom Roberto Oliveira Campos Neto – Presidente Bruno Serra Fernandes Carolina de Assis Barros Fabio Kanczuk Fernanda Feitosa Nechio João Manoel Pinho de Mello Maurício Costa de Moura Otávio Ribeiro Damaso Paulo Sérgio Neves de Souza Chefes de Departamento responsáveis por apresentações técnicas (presentes em 10/12) Alan da Silva Andrade Mendes – Departamento das Reservas Internacionais André Minella – Departamento de Estudos e Pe squisas (também presente em 11/12) André de Oliveira Amante – Departamento de O

In [22]:
filtered_df = df[df['inflation'] < 0.5]
print('Quantidade de sentenças resultantes:', len(filtered_df),'\n\n')
for x in filtered_df['sentenca'].head(20):
    print(x)
    print('\n----\n')

Quantidade de sentenças resultantes: 1553 


No cenário com trajetória de taxa de juros da pesquisa Focus e taxa de câmbio constante em R$4, 20/US$2, as projeções condicionais para a inflação situam-se ao redor de 4, 0% para 2019, 3, 7% para 2020 e 3, 7% para 2021.

----

B) Riscos em torno do cenário básico para a inflação 8.

----

Alguns membros avaliaram que os últimos dados de atividade e a maior eficiência do mercado de crédito e capitais podem resultar em uma redução da ociosidade mais rápida do que a ntevisto e, com isso, produzir pressão altista na inflação.

----

No horizonte relevante para a política monetária, o cenário com taxa Selic extraída da pesquisa Focus e taxa de câmbio constante produz inflação ligeiramente abaixo da meta para 2020 e ao redor da meta para 2021.

----

Dessa forma, a inflação acumulada em doze meses atingiu 7, 14% em janeiro (5, 59% em janeiro de 2014), com os preços livres aumentando 7, 01% (6, 65% em janeiro de 2014), e os administrados, 7, 55% (

In [25]:
filtered_df = df[df['inflation'] < 0.6]
print('Quantidade de sentenças resultantes:', len(filtered_df),'\n\n')
for x in filtered_df['sentenca'].head(20):
    print(x)
    print('\n----\n')

Quantidade de sentenças resultantes: 15044 


Ata da 22 7ª Reunião do Comitê de Política Monetária (Copom) do Banco Central do Brasil 10 e 11 de dezembro de 201 9 Banco Central do Brasil 227ª Reunião do Copom 2 Data: 10 e 11/12/2019 Local: Sala s de reuniões do 8º andar (10/12) e do 20º andar ( 11/12) do Edifício-Sede do Banco Central do Brasil – Brasília – DF Horário de início e de término das sessões: 10/12: 10h10 – 13h04; 15h13 – 19h38 11/12: 14h30 – 18h11 Presentes: Membros do Copom Roberto Oliveira Campos Neto – Presidente Bruno Serra Fernandes Carolina de Assis Barros Fabio Kanczuk Fernanda Feitosa Nechio João Manoel Pinho de Mello Maurício Costa de Moura Otávio Ribeiro Damaso Paulo Sérgio Neves de Souza Chefes de Departamento responsáveis por apresentações técnicas (presentes em 10/12) Alan da Silva Andrade Mendes – Departamento das Reservas Internacionais André Minella – Departamento de Estudos e Pe squisas (também presente em 11/12) André de Oliveira Amante – Departamento de O